In [7]:
import numpy as np
import pandas as pd
import math
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix, precision_score, recall_score

In [8]:
#Source of text:
#https://www.researchgate.net/publication/227988510_Automatic_Keyword_Extraction_from_Individual_Documents
sentences = []
with open('genia4ertraining/tokenized_sentences.txt','r') as f:
    lines = f.read().rstrip().split('\n')
    for line in lines:
        if len(line.split())==0:
            continue
        Text.append(line)

In [9]:
bowlist = []
for para in Text:
    bowlist.append(para.split(" "))
vocabulary = list(set().union(*bowlist))
processed_text = bowlist[0]+bowlist[1]+bowlist[2]+bowlist[3]+bowlist[4]+bowlist[5]+bowlist[6]+bowlist[7]+bowlist[8]+bowlist[9]
print(len(processed_text))

1434


In [10]:
vocab_len = len(vocabulary)

weighted_edge = np.zeros((vocab_len,vocab_len),dtype=np.float32)

score = np.zeros((vocab_len),dtype=np.float32)
window_size = 3
covered_coocurrences = []

for i in range(0,vocab_len):
    score[i]=1
    for j in range(0,vocab_len):
        if j==i:
            weighted_edge[i][j]=0
        else:
            for window_start in range(0,(len(processed_text)-window_size)):
                
                window_end = window_start+window_size
                
                window = processed_text[window_start:window_end]
                
                if (vocabulary[i] in window) and (vocabulary[j] in window):
                    
                    index_of_i = window_start + window.index(vocabulary[i])
                    index_of_j = window_start + window.index(vocabulary[j])
                    
                    # index_of_x is the absolute position of the xth term in the window 
                    # (counting from 0) 
                    # in the processed_text
                      
                    if [index_of_i,index_of_j] not in covered_coocurrences:
                        weighted_edge[i][j]+=1/math.fabs(index_of_i-index_of_j)
                        covered_coocurrences.append([index_of_i,index_of_j])


In [22]:
inout = np.zeros((vocab_len),dtype=np.float32)

for i in range(0,vocab_len):
    for j in range(0,vocab_len):
        inout[i]+=weighted_edge[i][j]

In [23]:
MAX_ITERATIONS = 50
d=0.85
threshold = 0.0001 #convergence threshold

for iter in range(0,MAX_ITERATIONS):
    prev_score = np.copy(score)
    
    for i in range(0,vocab_len):
        
        summation = 0
        for j in range(0,vocab_len):
            if weighted_edge[i][j] != 0:
                summation += (weighted_edge[i][j]/inout[j])*score[j]
                
        score[i] = (1-d) + d*(summation)
    
    if np.sum(np.fabs(prev_score-score)) <= threshold: #convergence condition
        print ("Converging at iteration "+str(iter)+"....")
        break


Converging at iteration 33....


In [6]:
newlist = list(zip(vocabulary, score))
#print(len(newlist))
newlist = sorted(newlist, key=lambda tup: tup[1],reverse=True)
preds = []
for i in range(0,len(newlist)):
    preds.append(list(newlist[i]))
#newdict = 

count = 0
for i in range(0,len(preds)):
    #print ("Score of "+newlist[i][0]+": "+str(newlist[i][1]))
    if count<358:
        preds[i][1] = 1
    else:
        preds[i][1] = 0
    count+=1
    


NameError: name 'score' is not defined

In [47]:
wrds = []
with open('4ersample.txt','r') as f:
    lines = f.read().splitlines()
    for i in lines:
        element = i.strip().split('\t')
        wrds.append(element)
        
predsfinal = sorted(preds, key=lambda tup: tup[0])
testfinal = sorted(wrds, key=lambda tup: tup[0])

In [48]:
preds = []
test = []
count = 0
for tup in predsfinal:
    if count<735:
        preds.append(tup[1])
        count+=1
for tup in testfinal:
    test.append(int(tup[1]))

preds = np.array(preds)
test = np.array(test)
print('acc',round(accuracy_score(preds,test),4))
print('f1',round(f1_score(preds,test),4))
print(confusion_matrix(preds,test))
print('prec',round(precision_score(preds,test),4))
print('rec',round(recall_score(preds,test),4))

acc 0.5605
f1 0.3084
[[340  95]
 [228  72]]
prec 0.4311
rec 0.24


In [54]:
phrases = []
with open('long_stopwords.txt','r') as fin:
    stopwords = fin.read().splitlines()
phrase = " "
for word in processed_text:
    if word in stopwords:
        if phrase!= " ":
            phrases.append(str(phrase).strip().split())
        phrase = " "
    
    elif word not in stopwords:
        phrase+=str(word)
        phrase+=" "

print ("Partitioned Phrases (Candidate Keyphrases): \n")
print (phrases)

Partitioned Phrases (Candidate Keyphrases): 

[]


In [15]:
unique_phrases = []

for phrase in phrases:
    if phrase not in unique_phrases:
        unique_phrases.append(phrase)

print "Unique Phrases (Candidate Keyphrases): \n"
print unique_phrases

Unique Phrases (Candidate Keyphrases): 

[['compatibility'], ['system'], ['linear', 'constraint'], ['set'], ['natural', 'number'], ['criterion'], ['linear', 'diophantine', 'equation'], ['strict', 'inequations'], ['nonstrict', 'inequations'], ['upper', 'bound'], ['component'], ['minimal', 'set'], ['solution'], ['algorithm'], ['construction'], ['minimal', 'generating', 'set'], ['type'], ['constructing'], ['minimal', 'supporting', 'set'], ['solving'], ['mixed', 'type']]


### Thinning the list of candidate-keyphrases.

Removing single word keyphrases-candidates that are present multi-word alternatives. 

In [16]:
for word in vocabulary:
    #print word
    for phrase in unique_phrases:
        if (word in phrase) and ([word] in unique_phrases) and (len(phrase)>1):
            #if len(phrase)>1 then the current phrase is multi-worded.
            #if the word in vocabulary is present in unique_phrases as a single-word-phrase
            # and at the same time present as a word within a multi-worded phrase,
            # then I will remove the single-word-phrase from the list.
            unique_phrases.remove([word])
            
print "Thinned Unique Phrases (Candidate Keyphrases): \n"
print unique_phrases    

Thinned Unique Phrases (Candidate Keyphrases): 

[['compatibility'], ['system'], ['linear', 'constraint'], ['natural', 'number'], ['criterion'], ['linear', 'diophantine', 'equation'], ['strict', 'inequations'], ['nonstrict', 'inequations'], ['upper', 'bound'], ['component'], ['minimal', 'set'], ['solution'], ['algorithm'], ['construction'], ['minimal', 'generating', 'set'], ['constructing'], ['minimal', 'supporting', 'set'], ['solving'], ['mixed', 'type']]


### Scoring Keyphrases

Scoring the phrases (candidate keyphrases) and building up a list of keyphrases\keywords
by listing untokenized versions of tokenized phrases\candidate-keyphrases.
Phrases are scored by adding the score of their members (words\text-units that were ranked by the graph algorithm)


In [17]:
phrase_scores = []
keywords = []
for phrase in unique_phrases:
    phrase_score=0
    keyword = ''
    for word in phrase:
        keyword += str(word)
        keyword += " "
        phrase_score+=score[vocabulary.index(word)]
    phrase_scores.append(phrase_score)
    keywords.append(keyword.strip())

i=0
for keyword in keywords:
    print "Keyword: '"+str(keyword)+"', Score: "+str(phrase_scores[i])
    i+=1

Keyword: 'compatibility', Score: 0.944583714008
Keyword: 'system', Score: 2.12031626701
Keyword: 'linear constraint', Score: 1.94610738754
Keyword: 'natural number', Score: 1.37661552429
Keyword: 'criterion', Score: 1.2255872488
Keyword: 'linear diophantine equation', Score: 2.83080631495
Keyword: 'strict inequations', Score: 2.13201224804
Keyword: 'nonstrict inequations', Score: 2.135455966
Keyword: 'upper bound', Score: 1.60279768705
Keyword: 'component', Score: 0.737640619278
Keyword: 'minimal set', Score: 4.05876886845
Keyword: 'solution', Score: 1.68319940567
Keyword: 'algorithm', Score: 1.19365406036
Keyword: 'construction', Score: 0.659808635712
Keyword: 'minimal generating set', Score: 4.71141409874
Keyword: 'constructing', Score: 0.66728836298
Keyword: 'minimal supporting set', Score: 4.71247345209
Keyword: 'solving', Score: 0.642318367958
Keyword: 'mixed type', Score: 1.31682945788


### Ranking Keyphrases

Ranking keyphrases based on their calculated scores. Displaying top keywords_num no. of keyphrases.

In [ ]:
sorted_index = np.flip(np.argsort(phrase_scores),0)

keywords_num = 10

print "Keywords:\n"

for i in xrange(0,keywords_num):
    print str(keywords[sorted_index[i]])+", ",